# 🌿 Green Space Change Detection in Tangerang Selatan, Indonesia
**Monitoring Vegetation Change from 2021 to 2025 Using Python & Google Earth Engine**

This project analyzes the dynamics of green space in **Tangerang Selatan**, Indonesia, by detecting **vegetation changes** using Sentinel-2 imagery and NDVI (Normalized Difference Vegetation Index). The project integrates **Python** and **Google Earth Engine (GEE)** to visualize where green space has **increased** or **decreased** between **2021 and May 2025**.

## 🛰️ Data
- **Satellite**: Sentinel-2 Surface Reflectance (`COPERNICUS/S2_SR`)
- **Source**: Google Earth Engine  
- **Period**: 2021 and May 2025
- **Cloud masking** applied using the **SCL (Scene Classification Layer)** band, which masks clouds, shadows, and cirrus.

## 🎯 Objectives
1. Generate NDVI-based vegetation maps for Tangerang Selatan that show the distribution of green spaces.
2. Identify areas with increased or decreased green space between 2021 and 2025.

## 🧪 Method Summary
- **NDVI** calculated for 2021 and 2025 using Sentinel-2 bands:  
  `NDVI = (B8 - B4) / (B8 + B4)`
- Define green space as **NDVI > 0.45**
- Calculate **NDVI difference**:  
  `NDVI_2025 - NDVI_2021`
- Thresholds used for change detection:  
  - **Increased vegetation**: ΔNDVI > 0.1  
  - **Decreased vegetation**: ΔNDVI < -0.1  
  - Changes only considered where **NDVI_2025 > 0.45** (i.e., still vegetation)

## 🗺️ Library Used
- `earthengine-api`  
- `folium`  
- `matplotlib`
- `ee`  
- `Python` + `Google Earth Engine` environment

## 📍 Study Area
- **Tangerang Selatan**, a rapidly urbanizing city in Banten, Indonesia

### Import Relevant Libraries

In [214]:
from IPython.display import Image
import matplotlib.pyplot as plt
import folium
import numpy as np
import ee

### Authenticate Google Earth Engine

In [215]:
# Authenticate your Google Earth Engine account (required once per session).
ee.Authenticate()

True

In [193]:
# Initialize the Earth Engine API after authentication.
ee.Initialize()

## Part 1 - Download and Display a Multispectral Image 🛰️

### Define Area of Interest (Tangerang Selatan)

We start by defining a central point in Tangerang Selatan using its coordinates, then create a 10 km buffer around it to generate a rectangular bounding box. Note: Google Maps provides coordinates in (latitude, longitude), but Earth Engine expects (longitude, latitude).

In [194]:
# Define coordinates for a central point in Tangerang Selatan
tangsel_coords = (-6.294132, 106.707277)  # (latitude, longitude)

# Create a point geometry (Earth Engine expects coordinates in (longitude, latitude) format)
tangsel_point = ee.Geometry.Point(tangsel_coords[::-1])

# Create a 10 km buffer around the point and extract the bounding box
tangsel = tangsel_point.buffer(10000).bounds()

### Download a Multispectral Sentinel-2 Image for the Study Area

In this step, we download a cloud-free multispectral image from the Sentinel-2 Surface Reflectance dataset using Google Earth Engine. The selected image represents the condition of green spaces in Tangerang Selatan from January to May 2025.

- **Dataset**: COPERNICUS/S2_SR (Surface Reflectance)
- **Date range**: 2025-01-01 to 2025-05-31
- **Cloud masking method**: SCL (Scene Classification Layer), masking out shadows, low/medium/high clouds, and cirrus
- **Filter criteria**: Less than 10% cloud coverage
- **Output**: A single clipped and cloud-masked image for analysis

In [195]:
# Load Sentinel-2 surface reflectance collection
s2_collection = ee.ImageCollection('COPERNICUS/S2_SR')

# Function to mask clouds and cloud shadows using the SCL band
def mask_s2_clouds(image):
    scl = image.select('SCL')
    
    # Define cloud and shadow classes to mask (based on ESA documentation)
    cloud_shadow = scl.eq(3)
    clouds_low = scl.eq(7)
    clouds_medium = scl.eq(8)
    clouds_high = scl.eq(9)
    cirrus = scl.eq(10)

    # Combine all unwanted classes into one mask
    mask = cloud_shadow.Or(clouds_low).Or(clouds_medium).Or(clouds_high).Or(cirrus).Not()
    
    # Apply mask and preserve image metadata
    return image.updateMask(mask).copyProperties(image, ["system:time_start"])

# Apply filters and masking to retrieve the least cloudy image for Jan–May 2025
s2_img = ee.Image(
    s2_collection
    .filterBounds(tangsel)  # Only images intersecting the AOI
    .filterDate('2025-01-01', '2025-05-31')  # Date range
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))  # Low cloud coverage
    .map(mask_s2_clouds)  # Apply cloud masking function
    .sort('CLOUDY_PIXEL_PERCENTAGE')  # Sort by cloudiness
    .first()  # Select the least cloudy image
    .clip(tangsel)  # Clip to AOI
)

### Displaying Google Earth Engine Images on Folium Maps

To visualize Earth Engine images interactively within a Python environment, I defined a helper function that integrates Earth Engine image tiles with Folium maps. This method converts Earth Engine images into map tiles and adds them as overlay layers on Folium maps.

The function uses Earth Engine’s `getMapId()` to fetch tile URLs and Folium’s `TileLayer` to render these tiles. This enables dynamic and interactive visualization of satellite data directly in Python notebooks.

This method is adapted from the official Google Earth Engine tutorial:  
https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-1


In [196]:
# Define a method for displaying Earth Engine image tiles to folium map
def add_ee_layer(self, ee_image_object, vis_params):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        overlay = True
    ).add_to(self)

# Add EE drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

### Create a True-Color Composite Image and Display on Map

For Sentinel-2, the main bands used for vegetation analysis include:

- **B2** = Blue  
- **B3** = Green  
- **B4** = Red  
- **B8** = Near Infrared (NIR)  

In [197]:
# Select Sentinel-2 bands for red (B4), green (B3), and blue (B2)
red = s2_img.select('B4')
green = s2_img.select('B3')
blue = s2_img.select('B2')
nir = s2_img.select('B8')

# Combine the selected bands into a single RGB image for true-color visualization
rgb_img = ee.Image.rgb(red, green, blue)

In [198]:
# Create the map object centered on Tangerang Selatan
m1 = folium.Map(location=tangsel_coords, zoom_start=11)

# Add the true-color composite image layer to the map with visualization parameters
m1.add_ee_layer(rgb_img, {
    'min': 0,
    'max': 3000
})

# Display the interactive map
m1

# Part 2. Calculating Normalised Difference Vegetation Index 🌳

The Normalised Difference Vegetation Index (NDVI) is a widely used indicator for measuring vegetation health and density.

It is calculated using the formula:

NDVI = (NIR - Red) / (NIR + Red)

Where:  
- **NIR** = Near Infrared band (Sentinel-2 band B8)  
- **Red** = Red band (Sentinel-2 band B4)

In [199]:
# Compute NDVI (Normalized Difference Vegetation Index) from Sentinel-2 bands
# NDVI = (NIR - Red) / (NIR + Red), highlighting vegetation health and density
ndvi = nir.subtract(red).divide(nir.add(red))
ndvi_img = ee.Image(ndvi)

In [200]:
# Create map centered on Tangerang Selatan
m2 = folium.Map(location=tangsel_coords, zoom_start=11)  

m2.add_ee_layer(ndvi_img, {                             # Add NDVI layer to map
    'min': -1,                                          # Set NDVI minimum value for visualization
    'max': 1,                                           # Set NDVI maximum value for visualization
    'palette': ['blue', 'white', 'green']               # Color palette: low to high vegetation
})

# Display the interactive map
m2  

## Mask to show only pixels with high NDVI  
* This creates a raster layer highlighting parks and other dense green areas  
* The threshold value of 0.45 is used to define “high” vegetation density 

In [157]:
# Mask NDVI to show only pixels with values > 0.45 (high vegetation)
ndvi_img_mask = ndvi_img.mask(ndvi_img.gt(0.45))  

# Create a folium map centered on Tangerang Selatan
m3 = folium.Map(location=tangsel_coords, zoom_start=11)  

# Add masked NDVI layer to the map
m3.add_ee_layer(ndvi_img_mask, {                    
    'min': -1,
    'max': 1,
    'palette': ['green']   # Use green color to highlight vegetation
})

# Display the interactive map
m3  

### Result

According to Sentinel-2 data in 2025, the map shows that green spaces such as parks and dense vegetation are mostly concentrated in the southwest part of Tangerang Selatan, while the northeast side has much less green cover. Note that the map is based on a bounding box and has not been clipped to the official administrative boundaries of Tangerang Selatan.

# Part 3 - Change Detection using Simple Difference 🕵️‍♀️

For this step, we use the least cloudy Sentinel-2 image from 2021 and the least cloudy image from January to May 2025.  
We detect changes in vegetation by calculating the simple difference between their NDVI values.

In [216]:
# Select the least cloudy Sentinel-2 image from 2021 for the Tangerang study area
s2_img_2021 = ee.Image(
    s2_collection
    .filterBounds(tangsel)  # Filter images overlapping the Tangerang area
    .filterDate('2021-01-01', '2021-12-31')  # Filter images from the year 2019
    .map(mask_s2_clouds)  # Apply cloud masking function
    .sort('CLOUDY_PIXEL_PERCENTAGE')  # Sort images by increasing cloud cover
    .first()  # Select the least cloudy image
    .clip(tangsel)  # Clip image to the study area boundary
)

# Select the least cloudy Sentinel-2 image from Jan to May 2025 for the same area
s2_img_2025 = ee.Image(
    s2_collection
    .filterBounds(tangsel)  # Filter images overlapping the Tangerang area
    .filterDate('2025-01-01', '2025-05-31')  # Filter images from early 2025
    .map(mask_s2_clouds)  # Apply cloud masking function
    .sort('CLOUDY_PIXEL_PERCENTAGE')  # Sort images by increasing cloud cover
    .first()  # Select the least cloudy image
    .clip(tangsel)  # Clip image to the study area boundary
)

In [217]:
def get_img_info(img):
    # Returns a formatted string with the image acquisition date and cloud cover percentage
    return (
        'Image from '
        + img.date().format('Y-M-d').getInfo() # Format the image date as Year-Month-Day
        + ' with the cloud cover of '
        + str(img.get('CLOUDY_PIXEL_PERCENTAGE').getInfo()) # Get the cloud cover value
        + '%'
    )

In [204]:
# Display metadata info of the 2019 Sentinel-2 image, including acquisition date and cloud cover percentage
get_img_info(s2_img_2021)

'Image from 2021-7-28 with the cloud cover of 0.223546%'

In [205]:
# Display metadata info of the 2025 Sentinel-2 image, including acquisition date and cloud cover percentage
get_img_info(s2_img_2025)

'Image from 2025-5-28 with the cloud cover of 7.425626%'

In [209]:
# Calculate NDVI for 2021 and 2025 images using Sentinel-2 NIR (B8) and Red (B4) bands
ndvi_2021 = s2_img_2021.normalizedDifference(['B8', 'B4'])
ndvi_2025 = s2_img_2025.normalizedDifference(['B8', 'B4'])

# Compute the difference in NDVI between 2025 and 2021 to detect vegetation changes
ndvi_diff = ndvi_2025.subtract(ndvi_2021)

### 2021 NDVI Image

In [207]:
# Mask NDVI to show only pixels with values > 0.45 (high vegetation)
ndvi_img_mask21 = ndvi_2021.mask(ndvi_2021.gt(0.45))  

# Create a folium map centered on Tangerang Selatan
m21 = folium.Map(location=tangsel_coords, zoom_start=11)  

# Add masked NDVI layer to the map
m21.add_ee_layer(ndvi_img_mask21, {                    
    'min': -1,
    'max': 1,
    'palette': ['green']   # Use green color to highlight vegetation
})

# Display the interactive map
m21

### 2025 NDVI Image

In [208]:
# Mask NDVI to show only pixels with values > 0.45 (high vegetation)
ndvi_img_mask25 = ndvi_2025.mask(ndvi_2025.gt(0.45))  

# Create a folium map centered on Tangerang Selatan
m25 = folium.Map(location=tangsel_coords, zoom_start=11)  

# Add masked NDVI layer to the map
m25.add_ee_layer(ndvi_img_mask25, {                    
    'min': -1,
    'max': 1,
    'palette': ['green']   # Use green color to highlight vegetation
})

# Display the interactive map
m25

### Green Space Change Map of Tangerang Selatan (2021–2025)

This map visualizes changes in vegetation cover based on NDVI difference between 2021 and 2025.

- **Blue areas** indicate significant **increase in vegetation** (NDVI difference > 0.2)
- **Red areas** indicate significant **decrease in vegetation** (NDVI difference < -0.2)

This helps identify patterns of urban development, reforestation, or land degradation across Tangerang Selatan.

In [211]:
# Create a folium map centered on Tangerang Selatan
m4 = folium.Map(location=tangsel_coords, zoom_start=11)

# Create a general vegetation mask using 2025 NDVI; threshold > 0.45 indicates green space
veg_mask = ndvi_2025.gt(0.45)

# Create a mask for areas with increased vegetation (NDVI change > 0.1)
veg_increase_mask = ndvi_diff.gt(0.1)

# Create a mask for areas with decreased vegetation (NDVI change < -0.1)
veg_decrease_mask = ndvi_diff.lt(-0.1)

# Display increased vegetation (in blue) only in vegetated areas
m4.add_ee_layer(
    ndvi_diff.updateMask(veg_mask).updateMask(veg_increase_mask),
    {'palette': ['blue']}
)

# Display decreased vegetation (in red) only in vegetated areas
m4.add_ee_layer(
    ndvi_diff.updateMask(veg_mask).updateMask(veg_decrease_mask),
    {'palette': ['red']}
)

# Show the interactive map
m4

In [213]:
# Define a pixel area image in square meters
pixel_area = ee.Image.pixelArea()

# Mask NDVI difference image with increase and decrease thresholds
increase_masked = ndvi_diff.updateMask(ndvi_diff.gt(0.2))   # Vegetation increase
decrease_masked = ndvi_diff.updateMask(ndvi_diff.lt(-0.2))  # Vegetation decrease

# Calculate total increased area in m²
increase_area = pixel_area.updateMask(increase_masked.mask()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=tangsel,
    scale=10,
    maxPixels=1e9
)

# Calculate total decreased area in m²
decrease_area = pixel_area.updateMask(decrease_masked.mask()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=tangsel,
    scale=10,
    maxPixels=1e9
)

# Convert to hectares
increase_area_ha = increase_area.getInfo()['area'] / 10_000
decrease_area_ha = decrease_area.getInfo()['area'] / 10_000

print(f"Total Green Space Increase: {increase_area_ha:.2f} ha")
print(f"Total Green Space Decrease: {decrease_area_ha:.2f} ha")

Total Green Space Increase: 308.76 ha
Total Green Space Decrease: 12213.76 ha


### Result

The analysis reveals a notable **decrease in green space** in Tangerang Selatan between 2019 and 2025. Using NDVI difference values, we defined a:

- **Decrease** as a drop of **at least 0.1** in NDVI value.
- **Increase** as a gain of **at least 0.1** in NDVI value,  
within areas that were previously vegetated (**NDVI > 0.45** in 2025).

Based on this definition:

- **12,213.76 hectares** of green space experienced a **decrease in vegetation density**, shown in **red** on the map.
- **308.76 hectares** saw a **significant increase in vegetation**, shown in **blue**, mostly in the **southwestern** part of the city.

> ⚠️ Note: A **decrease** does not necessarily indicate total vegetation loss or land-use change. It refers to a **reduction in NDVI**, implying lower vegetation density in the same locations.